<a href="https://colab.research.google.com/github/Sergrey5571/Sergrey5571/blob/main/%D0%9E%D0%B1%D1%83%D1%87_%D1%82%D0%B5%D1%81%D1%82_%D0%B2%D1%8B%D0%B1%D0%BE%D1%80%D0%BA%D0%B8_Ultra_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание Ultra Pro :

По подготовленным на занятии данным, создайте обучающую, тестовую и проверочную выборки. 

По сформированным данным необходимо обучить модель для предсказания цены машины. Оцените качество работы созданной сети, определив средний процент ошибки на проверочной выборке. (Для этого потребуется привести предсказанные моделью значения к первоначальному диапазону цен. Это можно сделать с помощью следующего метода:

predict_inverse = y_scaler.inverse_transform(predict).flatten()

где predict - результат предсказания модели).

Затем подсчитайте среднюю ошибку на тестовой выборке, среднюю цену и средний процент ошибки.

Рекомендации:

В качестве ошибки рекомендуется использовать среднеквадратическую ошибку (mse).
Метрику для данной задачи можно не использовать.
Последний слой модели должен иметь 1 нейрон.

In [ ]:
from tensorflow.keras.models import Sequential # НС прямого распространения
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization # Основные слои
from tensorflow.keras import utils # Утилиты для to_categorical
from tensorflow.keras.preprocessing import image # Для отрисовки изображения
from tensorflow.keras.optimizers import Adam, Adadelta # Алгоритмы оптимизации, для настройки скорости обучения
import numpy as np # Библиотека работы с массивами
import matplotlib.pyplot as plt # Отрисовка изображений
from sklearn.model_selection import train_test_split
from PIL import Image # Отрисовка изображений
import pandas as pd # Библиотека pandas
from google.colab import files # Импорт файлов
from sklearn.preprocessing import LabelEncoder, StandardScaler # Функции для нормализации данных
from sklearn import preprocessing # Пакет предварительной обработки данных

# Отрисовывать изображения в ноутбуке, а не в консоль или файл
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cars=pd.read_csv('/content/drive/MyDrive/cars_new.csv')

In [ ]:
cars[:5]

,mark,model,price,year,mileage,body,kpp,fuel,volume,power
0,kia,cerato,996000,2018,28000,седан,автомат,бензин,2.0,150.0
1,daewoo,nexia 1 поколение [2-й рестайлинг],140200,2012,60500,седан,механика,бензин,1.5,80.0
2,suzuki,jimny 3 поколение [рестайлинг],750000,2011,29000,внедорожник,автомат,бензин,1.3,85.0
3,bmw,x1 18 e84 [рестайлинг],970000,2014,49500,кроссовер,автомат,бензин,2.0,150.0
4,chevrolet,lacetti 1 поколение,205000,2007,151445,седан,механика,бензин,1.4,95.0


In [ ]:
cars['price'].max()

24500000

In [ ]:
print(cars.values.shape)

(70119, 10)


словарик:

In [ ]:
def create_dict(s):
  ret={}
  for _id, name in enumerate(s): # каждому ун. значению присвается индекс
    ret.update({name:_id})
  return ret  

преобразование :

In [ ]:
def to_ohe(value, d):
  arr=[0]*len(d)
  arr[d[value]]=1
  return arr

словари по колонкам:

In [ ]:
marks_dict= create_dict(set(cars['mark']))
models_dict= create_dict(set(cars['model']))
bodies_dict=create_dict(set(cars['body']))
kpps_dict=create_dict(set(cars['kpp']))
fuels_dict=create_dict(set(cars['fuel']))

цены на машины:

In [ ]:
prices = np.array(cars['price'], dtype=np.float)

In [ ]:
years= preprocessing.scale(cars['year'])
mileages=preprocessing.scale(cars['mileage'])
volumes= preprocessing.scale(cars['volume'])
powers= preprocessing.scale(cars['power'])

пустая выборка:

In [ ]:
x_train = []
y_train = []

#  по всем машинам
for _id, car in enumerate(np.array(cars)):
  # В y_train добавляем цену
  y_train.append(prices[_id]) 
  x_tr = to_ohe(car[0], marks_dict) + \
        to_ohe(car[1], models_dict) + \
        to_ohe(car[5], bodies_dict) + \
        to_ohe(car[6], kpps_dict) + \
        to_ohe(car[7], fuels_dict) + \
        [years[_id]] + \
        [mileages[_id]] + \
        [volumes[_id]] + \
        [powers[_id]]
  x_train.append(x_tr)

# Превращаем лист в numpy.array
x_train = np.array(x_train, dtype=np.float)
y_train = np.array(y_train, dtype=np.float)

хм.. 

In [ ]:
print(x_train.shape)
print(y_train.shape)

(70119, 3208)
(70119,)


опять новое в Python:

In [ ]:
y_scaler=StandardScaler() # из библиотеки Sklearn конструктор маштабирования
y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1,1)).flatten() #  не забывать скобки


In [ ]:
y_train_scaled.shape

(70119,)

In [ ]:
print(y_train_scaled[:15])

[ 0.75082432 -0.62135434  0.3563911   0.70913625 -0.51745486  0.73318706
  0.09824578 -0.04445567  1.27032173 -0.4132347  -0.38116696 -0.37315002
 -0.44530244 -0.08624635  0.19605239]


In [ ]:
x_train, x_test, y_train_scaled, y_test = train_test_split(x_train, y_train_scaled, test_size=0.2)

In [ ]:
model_ula = Sequential()
model_ula.add(Dense(300, activation = 'relu', input_shape = x_train.shape[1:]))
model_ula.add(Dense(100, activation = 'relu'))
model_ula.add(Dense(1))
model_ula.compile(optimizer = 'adam', loss = 'mse', metrics = ['mae'])
# поехали!
model_ula.fit(x_train, y_train_scaled, epochs = 20, batch_size = 32, validation_split = 0.1)

predict = model_ula.predict(x_test)

Epoch 1/20
1578/1578 [==============================] - 13s 8ms/step - loss: 0.1761 - mae: 0.1609 - val_loss: 0.0811 - val_mae: 0.1297
Epoch 2/20
1578/1578 [==============================] - 11s 7ms/step - loss: 0.1011 - mae: 0.1237 - val_loss: 0.0562 - val_mae: 0.1160
Epoch 3/20
1578/1578 [==============================] - 11s 7ms/step - loss: 0.0894 - mae: 0.1214 - val_loss: 0.0639 - val_mae: 0.1212
Epoch 4/20
1578/1578 [==============================] - 11s 7ms/step - loss: 0.0709 - mae: 0.1139 - val_loss: 0.0771 - val_mae: 0.1168
Epoch 5/20
1578/1578 [==============================] - 11s 7ms/step - loss: 0.0743 - mae: 0.1129 - val_loss: 0.1444 - val_mae: 0.1346
Epoch 6/20
1578/1578 [==============================] - 11s 7ms/step - loss: 0.0627 - mae: 0.1088 - val_loss: 0.0615 - val_mae: 0.1091
Epoch 7/20
1578/1578 [==============================] - 11s 7ms/step - loss: 0.0601 - mae: 0.1027 - val_loss: 0.0588 - val_mae: 0.1131
Epoch 8/20
1578/1578 [==============================] -

In [ ]:
print(predict)

[[-0.13142796]
 [-0.27163276]
 [ 0.86709684]
 ...
 [ 0.609318  ]
 [-0.17346595]
 [-0.65207815]]


In [ ]:
scores = model_ula.evaluate(x_test, y_test, verbose=1) #  оценка модельки на тестовой выборке

439/439 [==============================] - 1s 3ms/step - loss: 0.1315 - mae: 0.1112


In [ ]:
predict_inverse = y_scaler.inverse_transform(predict).flatten()  #обратная инверсия

дальше не очень понятно:

In [ ]:
predict_inverse.shape # размерность

(14024,)

In [ ]:
predict_inverse.mean() #средняя цена машины в модельке

537756.56

In [ ]:
predict_inverse.min() #  упс..... вот это как??? 

-17860.586

In [ ]:
print(predict_inverse.max())

18180858.0


In [ ]:
y_test.mean()

0.0023144084840627517

In [ ]:
inverse = y_scaler.inverse_transform(y_test).flatten() # инверсия не тронутой выборки

In [ ]:
inverse.shape

(14024,)

In [ ]:
inverse.mean() # средння цена в тестовой выборке

529169.5510553337

In [ ]:
x_oshibka=predict_inverse.mean()/inverse.mean() #

In [ ]:
print('Ср. ошибка:', round(x_oshibka *100-100, 4), '%')

Ср. ошибка: 1.6227 %
